In [2]:
#Importando Bibliotecas
import cv2
import vlc
import numpy as np
from tensorflow.keras.models import load_model
from keras.models import load_model

In [7]:
#Inicializando Variables

model = load_model('keras_model.h5')
p = vlc.MediaPlayer("wakeup.mp3")

TypeError: Error when deserializing class 'DepthwiseConv2D' using config={'name': 'expanded_conv_depthwise', 'trainable': True, 'dtype': 'float32', 'kernel_size': [3, 3], 'strides': [1, 1], 'padding': 'same', 'data_format': 'channels_last', 'dilation_rate': [1, 1], 'groups': 1, 'activation': 'linear', 'use_bias': False, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'bias_regularizer': None, 'activity_regularizer': None, 'bias_constraint': None, 'depth_multiplier': 1, 'depthwise_initializer': {'class_name': 'VarianceScaling', 'config': {'scale': 1, 'mode': 'fan_avg', 'distribution': 'uniform', 'seed': None}}, 'depthwise_regularizer': None, 'depthwise_constraint': None}.

Exception encountered: Unrecognized keyword arguments passed to DepthwiseConv2D: {'groups': 1}

In [4]:
import h5py

# Abre el archivo del modelo
f = h5py.File("keras_model.h5", mode="r+")
model_config_string = f.attrs.get("model_config")

# Elimina el argumento 'groups' si está presente
if model_config_string.find('"groups": 1,') != -1:
    model_config_string = model_config_string.replace('"groups": 1,', '')
    f.attrs.modify('model_config', model_config_string)
    f.flush()

f.close()

In [6]:

# Inicializando Variables
video_capture = cv2.VideoCapture(0)

# Cargar el clasificador de ojos izquierdo
eyeLeft = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_lefteye_2splits.xml')
eyeRight = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_righteye_2splits.xml')
# Verificar que los clasificadores se hayan cargado correctamente
if eyeLeft.empty():
    print("Error al cargar el clasificador de ojos izquierdos.")
if eyeRight.empty():
    print("Error al cargar el clasificador de ojos derechos.")

# Cargar el modelo
model = load_model('keras_model.h5')
p = vlc.MediaPlayer("wakeup.mp3")

# Variables para almacenar coordenadas de los ojos
left_x, left_y, left_w, left_h = 0, 0, 0, 0
right_x, right_y, right_w, right_h = 0, 0, 0, 0
contador = 0
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

music_playing = False
cv2.namedWindow('video', cv2.WINDOW_NORMAL)
cv2.resizeWindow('video', 800, 720)
#Ejecutando Video
while True:
    ret, frame = video_capture.read()
    height,width = frame.shape[:2]
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    #Mostrando Contador
    cv2.rectangle(frame, (0, 0), (width, int(height*0.1)), (0,0,0), -1)
    cv2.putText(frame,'Contador: '+str(contador),(int(width*0.65), int(height*0.08)), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2)
    
    #Identificando el Ojo Derecho
    ojo_der = eyeRight.detectMultiScale(
        gray,
        scaleFactor=1.15,
        minNeighbors=3,
        minSize=(30, 30)
    )
    for (x, y, w, h) in ojo_der:
        #cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        right_x, right_y, right_w, right_h = x, y, w, h
        break
    
    #Identificando el Ojo Izquierdo
    ojo_izq = eyeLeft.detectMultiScale(
        gray,
        scaleFactor=1.15,
        minNeighbors=3,
        minSize=(30, 30)
    )
    for (x, y, w, h) in ojo_izq:
        #cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        left_x, left_y, left_w, left_h = x, y, w, h
        break
    
    #Identificando Coordenadas x,y iniciales y finales para extraer la foto de los ojos
    if (left_x > right_x):
        start_x, end_x = right_x, (left_x+left_w)
    else: 
        start_x, end_x = left_x, (right_x+right_w)

    if (left_y > right_y):
        start_y, end_y = right_y, (left_y+left_h)
    else:
        start_y, end_y = left_y, (right_y+right_h)
    
    
    #Algoritmo de deteccion de sueño
    if ((end_x-start_x)>120 and (end_y-start_y)<200):
        start_x, start_y, end_x, end_y = start_x-30, start_y-50, end_x+30, end_y+50
        cv2.rectangle(frame, (start_x, start_y), (end_x, end_y), (0, 255, 0), 2)
        img = frame[start_y:end_y, start_x:end_x]
        imagen = cv2.resize(img, (224, 224))
        imagen_normalizada = (imagen.astype(np.float32) / 127.0) - 1
        data[0] = imagen_normalizada
        prediction = model.predict(data)
        
        if (list(prediction)[0][1]>=0.95):
            cv2.putText(frame,'Durmiendo : '+str(round(list(prediction)[0][1],3)),(10, int(height*0.08)), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2)
            contador+=1
            if not music_playing and contador >= 15:
                music_playing = True
                p.play()
        
        if (list(prediction)[0][0]>=0.95):
            cv2.putText(frame,'Despierto',(10, int(height*0.08)), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2)
            contador-=1
            if music_playing:  # Apaga la música si está sonando
                music_playing = False
                p.stop()
                
        contador = max(0, min(contador, 15))
            
    #Mostrar el video de la webcam
    cv2.imshow('video', frame)

    #Tecla de salida - acaba la transmisión
    if cv2.waitKey(1) & 0xFF == ord('q'):
        video_capture.release()
        cv2.destroyAllWindows()
        break

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


: 